In [1]:
import pyspark
import zipfile
import databricks.koalas as ks
import sys
sys.path.insert(0, '../')
from util import util_data_access
from pyspark.sql import SparkSession
import tqdm
import pandas as pd 
import os, glob
import re
from tqdm.auto import tqdm

In [5]:
## download spacy model 
# download data
util_data_access.download_file('en_core_web_lg.tar.gz', 'edit-pathways/spacy/en_core_web_lg.tar.gz')
! tar -xf en_core_web_lg.tar.gz
! mv en_core_web_lg-2.3.1 en_core_web_lg

# Turn On Spark

In [2]:
# turn on spark
spark = (
    SparkSession.builder \
      .config("spark.executor.instances", "50") \
      .config("spark.driver.memory", "15g") \
      .getOrCreate()
)
sc = spark.sparkContext
# set arrow to true
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
ks.set_option('compute.default_index_type', 'distributed')
ks.set_option('compute.shortcut_limit', 1) # Koalas will apply pyspark 

In [3]:
spark

# Get Data and Read In

In [4]:
import sqlite3
conn = sqlite3.connect('newssniffer-nytimes.db')
df = pd.read_sql('''
    SELECT * from entryversion 
    WHERE entry_id IN (SELECT distinct entry_id FROM entryversion LIMIT 200)
''', con=conn)

In [5]:
sdf = spark.createDataFrame(df)

# Basic Functions

In [6]:
import spacy
from pyspark import SparkConf, SparkContext, SparkFiles
sc.addFile("en_core_web_lg.tar.gz")
sys.path.insert(0,SparkFiles.getRootDirectory())

nlp = None
def get_nlp():
    import spacy
    global nlp
    nlp = None
    # untar
    tar_fn = 'en_core_web_lg.tar.gz'
    dirname = 'en_core_web_lg'
    if os.path.exists(tar_fn) and not os.path.exists(dirname):
        import tarfile, glob
        with tarfile.open(tar_fn) as tar:
            tar.extractall(path='en_core_web_lg')
#         for fn in glob.glob(os.path.join(dirname, '*', '*')):
#             os.rename(fn, os.path.join(dirname, os.path.basename(fn)))
        
    # unload 
    if not nlp:
        nlp = spacy.load("en_core_web_lg/en_core_web_lg-2.3.1", disable=["tagger" "ner"])
    return nlp

In [9]:
from pyspark.sql.functions import col, pandas_udf

In [17]:
lang="en"
def spacy_tokenize(x):
    # Note this is expensive, in practice you would use something like SpacyMagic, see footnote for link; which caches
    # spacy.load so it isn’t happening multiple times
    nlp = get_nlp()
    doc = nlp(x)
    return [token.text for token in doc]
 
spark.udf.register("get_nlp", get_nlp)
spark.udf.register("tokenize", spacy_tokenize)

<function __main__.spacy_tokenize(x)>

In [19]:
kdf = ks.from_pandas(df)

In [21]:
t = kdf['summary'].apply(spacy_tokenize)

In [24]:
from tqdm.auto import tqdm

In [11]:
from pyspark.sql.types import LongType
from pyspark.sql.functions import col, pandas_udf, struct, PandasUDFType

In [12]:
@pandas_udf("integer", PandasUDFType.SCALAR)
def pandas_tokenize(x):
    return x.apply(spacy_tokenize)
tokenize_pandas = spark.udf.register("tokenize_pandas", pandas_tokenize)

In [18]:
from pyspark.sql.functions import split, explode
tokenized = sdf.select(explode(spacy_tokenize(sdf.summary)))

TypeError: object of type 'method' has no len()

In [17]:
sdf.select(tokenize_pandas(col("summary"))).show()

Py4JJavaError: An error occurred while calling o245.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, 10.178.189.104, executor 6): ExecutorLostFailure (executor 6 exited caused by one of the running tasks) Reason: 
The executor with id 6 exited with exit code 143.
The API gave the following brief reason: null
The API gave the following message: null
The API gave the following container statuses:

ContainerStatus(containerID=docker://17bf58cb70cb0615997aac4cbad425543633b4800ac359e5388204327d1199e8, image=artifactory.inf.bloomberg.com/ds/frameworks/spark-2.4-python-3.7:1.129, imageID=docker-pullable://artifactory.inf.bloomberg.com/ds/frameworks/spark-2.4-python-3.7@sha256:5a3539a36f8e601dfd78a4151c62e3d3922dfc92610bf33d7c6dc89fe5b1dbb2, lastState=ContainerState(running=null, terminated=null, waiting=null, additionalProperties={}), name=executor, ready=false, restartC